实例演示：
1. 将一个大的excel等份拆成多个excel
2. 将小excel合并成一个大的excel并标记来源


In [1]:
import os
# print(os.getcwd())
# 
work_dir="./course_datas/c15_excel_split_merge"  # 等同于root
split_dir=f"{work_dir}/拆分"
if not os.path.exists(split_dir):  # 如果下面的一个文件不存在，就建立一个
    os.mkdir(split_dir)

In [2]:
import pandas as pd
df=pd.read_excel(f"{work_dir}/crazyant_blog_articles_source.xlsx")

In [3]:
df.head()

,id,title,tags
0,2585,Tensorflow怎样接收变长列表特征,"python,tensorflow,特征工程"
1,2583,Pandas实现数据的合并concat,"pandas,python,数据分析"
2,2574,Pandas的Index索引有什么用途？,"pandas,python,数据分析"
3,2564,机器学习常用数据集大全,"python,机器学习"
4,2561,一个数据科学家的修炼路径,数据分析


In [4]:
df.shape

(258, 3)

将以上excel拆分成六个。保存到六个文件中去，


In [5]:
user_names=list("ABCDEF")
total_row=df.shape[0]
split_size=total_row//len(user_names)
if total_row % len(user_names)!=0:
    split_size+=1
split_size

43

In [8]:
df_subs=[]
for idx,user_name in enumerate(user_names):
    start=idx*split_size
    end=start+split_size
    df_sub=df.iloc[start:end]
    df_subs.append((idx,user_name,df_sub))  # 将idx,name和 负责的表格区域分割好
   
for idx,username,df_sub in df_subs:
    print(idx,username)
    file_name=f"{split_dir}/crazyant_blog_articles_{username}_{idx}.xlsx"
    df_sub.to_excel(file_name,index=False)

0 A
1 B
2 C
3 D
4 E
5 F


### 合并多个小excel到一个大excel
+ 遍历文件夹，找到所有的excel
+ 分别读取到dataframe,给每个df添加一列用于标记来源
+ 使用pd.concat 进行批量合并
+ 将合并后的dataframe输出到excel

In [9]:
# 遍历文件按夹
names=[]
for i in os.listdir(split_dir):
    names.append(i)
names

['crazyant_blog_articles_A_0.xlsx',
 'crazyant_blog_articles_B_1.xlsx',
 'crazyant_blog_articles_C_2.xlsx',
 'crazyant_blog_articles_D_3.xlsx',
 'crazyant_blog_articles_E_4.xlsx',
 'crazyant_blog_articles_F_5.xlsx']

In [25]:
all_content_list=[]
# 读取文件存为dataframe格式
for read_name in names:
    read_path=f"{split_dir}/{read_name}"
    print(read_path)
    read_content=pd.read_excel(read_path)  # 得到读取的dataframe
    # 提取用户名，知道是谁负责的
    do_name=read_name.split("_")[-2]
    print(do_name)
    read_content.loc[:,"who do this"]=do_name  # 新增一列
    print(type(read_content))
    all_content_list.append(read_content)


./course_datas/c15_excel_split_merge/拆分/crazyant_blog_articles_A_0.xlsx
A
<class 'pandas.core.frame.DataFrame'>
./course_datas/c15_excel_split_merge/拆分/crazyant_blog_articles_B_1.xlsx
B
<class 'pandas.core.frame.DataFrame'>
./course_datas/c15_excel_split_merge/拆分/crazyant_blog_articles_C_2.xlsx
C
<class 'pandas.core.frame.DataFrame'>
./course_datas/c15_excel_split_merge/拆分/crazyant_blog_articles_D_3.xlsx
D
<class 'pandas.core.frame.DataFrame'>
./course_datas/c15_excel_split_merge/拆分/crazyant_blog_articles_E_4.xlsx
E
<class 'pandas.core.frame.DataFrame'>
./course_datas/c15_excel_split_merge/拆分/crazyant_blog_articles_F_5.xlsx
F
<class 'pandas.core.frame.DataFrame'>


### 使用pd.concat进行合并
+ concat的对象是一个dataframe的列表\[dataframe,dataframe,datafram\]

In [26]:
df_merged=pd.concat(all_content_list)

In [28]:
df_merged.head()

,id,title,tags,who do this
0,2585,Tensorflow怎样接收变长列表特征,"python,tensorflow,特征工程",A
1,2583,Pandas实现数据的合并concat,"pandas,python,数据分析",A
2,2574,Pandas的Index索引有什么用途？,"pandas,python,数据分析",A
3,2564,机器学习常用数据集大全,"python,机器学习",A
4,2561,一个数据科学家的修炼路径,数据分析,A


In [29]:
df_merged.shape

(258, 4)

In [33]:
df_merged["who do this"].value_counts()

D    43
B    43
F    43
E    43
C    43
A    43
Name: who do this, dtype: int64

In [36]:
df_merged.to_excel(f"{split_dir}/merged_article.xlsx",index=False) # 合并时候不需要index信息